<a href="https://colab.research.google.com/github/fedyasa/Task1/blob/main/FedorSergeev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = (pd.read_csv('train_regression.csv', header = None).iloc[: ,:-1]).to_numpy()  
train_targets = (pd.read_csv('train_regression.csv', header = None).iloc[: ,-1]).to_numpy()  

test_data = (pd.read_csv('test_regression.csv', header = None)).to_numpy()

In [3]:
print(train_data.shape)
print(test_data.shape)

print(train_targets.shape)

(1000, 256)
(100, 256)
(1000,)


In [8]:
mean = train_data.mean(axis = 0)
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std

test_data -= mean
test_data /= std

In [9]:
from keras import models
from keras import layers

In [14]:
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation = 'relu',
                         input_shape = (train_data.shape[1], )))
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
  return model

In [ ]:
k = 5
num_val_samples = len(train_data) // k
num_epochs = 500
all_mae_histories = []

for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples : (i + 1) * num_val_samples]

  partial_train_data = np.concatenate([train_data[: i * num_val_samples],
                                       train_data[(i + 1) * num_val_samples :]],
                                       axis = 0)
  partial_train_targets = np.concatenate([train_targets[: i * num_val_samples],
                                          train_targets[(i + 1) * num_val_samples :]],
                                          axis = 0)
  
  model = build_model()
  history = model.fit(partial_train_data, partial_train_targets,
                      validation_data = (val_data, val_targets),
                      epochs = num_epochs, batch_size = 1)
  mae_history = history.history['val_mae']
  all_mae_histories.append(mae_history)

processing fold # 0
Epoch 1/500
800/800 [==============================] - 2s 3ms/step - loss: 1429.5446 - mae: 24.4374 - val_loss: 105.2828 - val_mae: 8.1176
Epoch 2/500
800/800 [==============================] - 2s 2ms/step - loss: 98.9206 - mae: 7.4714 - val_loss: 123.1032 - val_mae: 7.9239
Epoch 3/500
800/800 [==============================] - 2s 3ms/step - loss: 78.6718 - mae: 6.3383 - val_loss: 111.2778 - val_mae: 7.4602
Epoch 4/500
800/800 [==============================] - 3s 3ms/step - loss: 61.0324 - mae: 5.6473 - val_loss: 102.2201 - val_mae: 7.2448
Epoch 5/500
800/800 [==============================] - 2s 2ms/step - loss: 56.5565 - mae: 5.2352 - val_loss: 98.4909 - val_mae: 7.1876
Epoch 6/500
800/800 [==============================] - 2s 2ms/step - loss: 52.9659 - mae: 5.0217 - val_loss: 102.3027 - val_mae: 7.1939
Epoch 7/500
800/800 [==============================] - 2s 2ms/step - loss: 48.6266 - mae: 4.9553 - val_loss: 84.2285 - val_mae: 6.6167
Epoch 8/500
800/800 [======

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()